# scrap

## Getting data from 77kaoded.com

In [ ]:
pip install scrapy

In [ ]:
from gc import callbacks
import scrapy
import json
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.crawler import CrawlerProcess

In [ ]:
class kaodedSpider(scrapy.Spider):
    name = "kaoded"
    start_urls = ['https://www.77kaoded.com/?s=%E0%B8%AD%E0%B8%B8%E0%B8%9A%E0%B8%B1%E0%B8%95%E0%B8%B4%E0%B9%80%E0%B8%AB%E0%B8%95%E0%B8%B8']
    
    custom_settings = {
        'FEEDS': {'kaoded.json': { 'format': 'json'}},
        'DEPTH_LIMIT': 10
    }
    
    def parse(self, response):
        for news in response.css('div.content-item.-card.card.-side'):
            yield {
                'topic_acc': news.css('div.info h3.entry-title.title a::text').get(),
                'date_published': news.css('div.info span.posted-on time.entry-date.published::text').get(),
                'province_acc': news.css('div.pic a.button.button-primary.button-province.button-small::text').get(),
                'link_url': news.css('h3.entry-title.title a::attr(href)').get(),
            }    

        next_page = response.css('div.content-pagination a.next.page-numbers::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [ ]:
process = CrawlerProcess()
process.crawl(kaodedSpider)
process.start()

## Upload data to DataPlatform

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_raw_id = 64

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='kaoded.json')